<a href="https://colab.research.google.com/github/Akashsky123/Deep-Learning-Lab/blob/main/Experiment2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**IMPORT NUMPY LIBRARY**

In [1]:
import numpy as np
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder


**LOAD MNIST**

In [4]:
# Load MNIST
X, y = fetch_openml('mnist_784', version=1, return_X_y=True)

X = X / 255.0   # normalize
y = y.astype(int)

# One-hot encode labels
encoder = OneHotEncoder(sparse_output=False)
y = encoder.fit_transform(y.to_numpy().reshape(-1,1)) # Convert Series to NumPy array before reshaping

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)


**Build Neural Network from Scratch(Activation Functions)**

In [5]:
def relu(z):
    return np.maximum(0, z)

def relu_derivative(z):
    return z > 0

def softmax(z):
    exp = np.exp(z - np.max(z, axis=1, keepdims=True))
    return exp / np.sum(exp, axis=1, keepdims=True)


**Initialize Weights**

In [6]:
np.random.seed(42)

input_size = 784
hidden_size = 128
output_size = 10

W1 = np.random.randn(input_size, hidden_size) * 0.01
b1 = np.zeros((1, hidden_size))

W2 = np.random.randn(hidden_size, output_size) * 0.01
b2 = np.zeros((1, output_size))


**Forward** **Propagation**

In [7]:
def forward(X):
    z1 = np.dot(X, W1) + b1
    a1 = relu(z1)

    z2 = np.dot(a1, W2) + b2
    a2 = softmax(z2)

    return z1, a1, z2, a2


**Loss Function (Cross Entropy)**

In [ ]:
def loss(y_true, y_pred):
    return -np.mean(np.sum(y_true * np.log(y_pred + 1e-8), axis=1))


**Backpropagation**

In [8]:
def backward(X, y, z1, a1, a2):
    global W1, b1, W2, b2

    m = X.shape[0]

    dz2 = a2 - y
    dW2 = np.dot(a1.T, dz2) / m
    db2 = np.sum(dz2, axis=0, keepdims=True) / m

    da1 = np.dot(dz2, W2.T)
    dz1 = da1 * relu_derivative(z1)
    dW1 = np.dot(X.T, dz1) / m
    db1 = np.sum(dz1, axis=0, keepdims=True) / m

    return dW1, db1, dW2, db2


**Training** **Loop**

In [10]:
def loss(y_true, y_pred):
    return -np.mean(np.sum(y_true * np.log(y_pred + 1e-8), axis=1))

lr = 0.1
epochs = 20
batch_size = 64

for epoch in range(epochs):
    for i in range(0, X_train.shape[0], batch_size):
        X_batch = X_train[i:i+batch_size]
        y_batch = y_train[i:i+batch_size]

        z1, a1, z2, a2 = forward(X_batch)
        l = loss(y_batch, a2)

        dW1, db1, dW2, db2 = backward(X_batch, y_batch, z1, a1, a2)

        W1 -= lr * dW1
        b1 -= lr * db1
        W2 -= lr * dW2
        b2 -= lr * db2

    print(f"Epoch {epoch+1}, Loss: {l:.4f}")


Epoch 1, Loss: 0.3750
Epoch 2, Loss: 0.2235
Epoch 3, Loss: 0.1552
Epoch 4, Loss: 0.1160
Epoch 5, Loss: 0.0925
Epoch 6, Loss: 0.0797
Epoch 7, Loss: 0.0729
Epoch 8, Loss: 0.0675
Epoch 9, Loss: 0.0622
Epoch 10, Loss: 0.0561
Epoch 11, Loss: 0.0526
Epoch 12, Loss: 0.0486
Epoch 13, Loss: 0.0453
Epoch 14, Loss: 0.0424
Epoch 15, Loss: 0.0395
Epoch 16, Loss: 0.0370
Epoch 17, Loss: 0.0353
Epoch 18, Loss: 0.0319
Epoch 19, Loss: 0.0307
Epoch 20, Loss: 0.0293


**Prediction & Accuracy**

In [11]:
def predict(X):
    _, _, _, a2 = forward(X)
    return np.argmax(a2, axis=1)

y_pred = predict(X_test)
y_true = np.argmax(y_test, axis=1)

accuracy = np.mean(y_pred == y_true)
print("Test Accuracy:", accuracy)


Test Accuracy: 0.9736428571428571
